In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torchvision
import argparse
from torch.autograd import Variable
import torch.utils.data as data

from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from data import v2, v1, detection_collate
#from data import MSCOCODetection, COCOAnnotationTransform
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time
import sys

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
train_sets = [('2007', 'trainval'), ('2012', 'trainval')]
# train_sets = 'train'
ssd_dim = 300  # only support 300 now
means = (104, 117, 123)  # only support voc now
num_classes = len(VOC_CLASSES) + 1
batch_size = 1
accum_batch_size = 1
iter_size = accum_batch_size / batch_size
max_iter = 120000
weight_decay = 0.0005
stepvalues = (80000, 100000, 120000)
gamma = 0.1
momentum = 0.9
cuda = False


version = "v2"
basenet = "vgg16_reducedfc.pth"
jaccard_threshold=0.5
start_iter=0
lr=1e-3
save_folder = "./weights/"
basenet = "vgg16_reducedfc.pth"

if cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

cfg = (v1, v2)[version == 'v2']

In [3]:
print('Loading Dataset...')

dataset = VOCDetection(VOCroot, train_sets,
                       SSDAugmentation(ssd_dim, means),
                       AnnotationTransform())
print("Dataset Loaded!")

data_loader = data.DataLoader(dataset, batch_size, num_workers=0,
                             shuffle=True, collate_fn=detection_collate,
                             pin_memory=cuda)

Loading Dataset...
Dataset Loaded!


In [4]:
batch = iter(data_loader)
_, targets = next(batch)
print(type(targets[0]))
%time tmp = np.concatenate(list(map(lambda x: x.numpy(), targets)))
print(tmp[:10,:])

<class 'torch.FloatTensor'>
CPU times: user 32 µs, sys: 11 µs, total: 43 µs
Wall time: 51.3 µs
[[  0.1572327    0.           0.77672958   1.          14.        ]]


In [5]:
type(tmp[2][3])

IndexError: index 2 is out of bounds for axis 0 with size 1

In [ ]:
xtmp = tmp[:,2]-tmp[:,0]
ytmp = tmp[:,3]-tmp[:,1]
area = xtmp*ytmp
print(min(area)," ",max(area))

In [6]:
def func(dataloader):
    batch = iter(data_loader)
    MIN, MAX = 100, -10
    i=1
    # for i in range(10):
    try:
        while True:
            _, targets = next(batch)
            # print(type(targets[0]))
            tmp = np.concatenate(list(map(lambda x: x.numpy(), targets)))
            # print(tmp[:10,:])
            xtmp = tmp[:,2]-tmp[:,0]
            ytmp = tmp[:,3]-tmp[:,1]
            area = xtmp*ytmp
            tmin = min(area)
            tmax = max(area)
            MIN = min(MIN,tmin)
            MAX = max(MAX,tmax)
            i=i+1
            if i % 100 == 0:
                print("batch: ",i,"|| min: ",tmin," || max: ",tmax)
    except:
        print("Overall min: ",MIN," || max: ",MAX)
    finally:
        del batch

In [7]:
print(len(dataset))

16551


In [8]:
def func1(dataloader):
    batch = iter(data_loader)
    MIN, MAX = 100, -10
    for img, targets in batch:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-8-99a4bf559385>, line 5)

In [9]:
func(data_loader)

batch:  100 || min:  0.026496  || max:  0.151557
batch:  200 || min:  1.0  || max:  1.0
batch:  300 || min:  0.509697  || max:  0.760797
batch:  400 || min:  0.180376  || max:  0.458649
batch:  500 || min:  0.168842  || max:  0.561155
batch:  600 || min:  0.463299  || max:  0.463299
batch:  700 || min:  0.326531  || max:  0.326531
batch:  800 || min:  0.129805  || max:  0.129805
batch:  900 || min:  0.477778  || max:  0.477778
batch:  1000 || min:  0.0551036  || max:  0.0551036
batch:  1100 || min:  1.0  || max:  1.0
batch:  1200 || min:  0.0981079  || max:  0.0981079
batch:  1300 || min:  0.927882  || max:  0.927882
batch:  1400 || min:  0.563535  || max:  0.563535
batch:  1500 || min:  0.00266407  || max:  0.232779
batch:  1600 || min:  0.0389065  || max:  0.300818
batch:  1700 || min:  0.00508802  || max:  0.00848824
batch:  1800 || min:  0.275544  || max:  0.449502
batch:  1900 || min:  0.0155846  || max:  0.0332719
batch:  2000 || min:  0.000720605  || max:  0.0521747
batch:  2100

batch:  16400 || min:  0.00277734  || max:  0.0410875
batch:  16500 || min:  0.106231  || max:  0.266254
Overall min:  3.55524e-05  || max:  1.0
